<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2016/Chapter_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 16.1 What is a regular expression?

In [1]:
open("textfile", 'w').write("""this is a file with hello
on 2 of 3 lines
hello.""")

import re
regexp = re.compile("hello")
count = 0
file = open("textfile", 'r')
for line in file.readlines():
    if regexp.search(line):
        count = count + 1
file.close()
print(count)

2


# 16.3 Regular expressions and raw strings

## 16.3.1 Raw strings to the rescue

In [2]:
r"Hello" == "Hello"

True

In [3]:
r"\the" == "\\the"

True

In [4]:
r"\the" == "\the"

False

In [5]:
print(r"\the")

\the


In [6]:
print("\the")

	he


# 16.4 Extracting matched text from strings

In [7]:
open("textfile", 'w').write("""Lopez, Maria: 111-555-2222
Jackson, Fred: 555-3333""")

import re
regexp = re.compile(r"[-a-zA-Z]+,"
                    r" [-a-zA-Z]+"
                    r"( [-a-zA-Z]+)?"
                    r": (\d{3}-)?\d{3}-\d{4}"
                   )
file = open("textfile", 'r')
for line in file.readlines():
    if regexp.search(line):
        print("Yeah, I found a line with a name and number. So what?")
        break
file.close()

Yeah, I found a line with a name and number. So what?


In [8]:
import re
regexp = re.compile(r"(?P<last>[-a-zA-Z]+),"
                    r" (?P<first>[-a-zA-Z]+)"
                    r"( (?P<middle>([-a-zA-Z]+)))?"
                    r": (?P<phone>((\d{3}-)?\d{3}-\d{4}))"
                   )
file = open("textfile", 'r')
for line in file.readlines():
    result = regexp.search(line)
    if result == None:
        print("Oops, I don't think this is a record")
    else:
        last_name = result.group('last')
        first_name = result.group('first')
        middle_name = result.group('middle')
        if middle_name == None:
            	middle_name = ""
        phone_number = result.group('phone')
        print(f"Name: {first_name} {middle_name} {last_name} Number: {phone_number}")
file.close()

Name: Maria  Lopez Number: 111-555-2222
Name: Fred  Jackson Number: 555-3333


### Try This: Extracting matched text

Making international calls usually requires a + and the country code. Assuming that the country code is two digits, how would you modify the code above to extract the + and the country code as part of the number? (Again, not all numbers have a country code.) How would you make the code handle country codes of one to three digits?

In [ ]:
# @title
print(re.match(r": (?P<phone>(\+\d{2}-)?(\d\d\d-)?\d\d\d-\d\d\d\d)", ": +01-111-222-3333"))

# or
print(re.match(r": (?P<phone>(\+\d{2}-)?(\d{3}-)?\d{3}-\d{4})", ": +01-111-222-3333"))

# or
# For one- to three-digit country codes:
print(re.match(r": (?P<phone>(\+\d{1,3}-)?(\d{3}-)?\d{3}-\d{4})", ": +011-111-222-3333"))

# 16.5 Substituting text with regular expressions

In [9]:
import re
string = "If the the problem is textual, use the the re module"
pattern = r"the the"
regexp = re.compile(pattern)
regexp.sub("the", string)

'If the problem is textual, use the re module'

## 16.5.1 Using a function with sub

In [10]:
import re
int_string = "1 2 3 4 5"
def int_match_to_float(match_obj):
    return(match_obj.group('num') + ".0")

pattern = r"(?P<num>[0-9]+)"
regexp = re.compile(pattern)
regexp.sub(int_match_to_float, int_string)

'1.0 2.0 3.0 4.0 5.0'

In [11]:
import re
int_string = "1 2 3 4 5 Python"
def int_match_to_float(match_obj):
    return(match_obj.group('num') + ".0")

pattern = r"(?P<num>[0-9]+)"
regexp = re.compile(pattern)
regexp.sub(int_match_to_float, int_string)

'1.0 2.0 3.0 4.0 5.0 Python'

### Try This: Replacing text

In the checkpoint in section 16.4, you extended a phone-number regular expression to also recognize a country code. How would you use a function to make any numbers that didn't have a country code now have +1 (the country code for the United States and Canada)?

In [ ]:
# @title
def add_code(match_obj):
    return("+1 "+match_obj.group('phone'))

re.sub(r"(?P<phone>(\d{3}-)?\d{3}-\d{4})", add_code, "111-222-3333")

# 16.6 Lab 16: Phone-Number normalizer

In the United States and Canada, phone numbers consist of ten digits, usually separated into a three-digit area code, a three-digit exchange code, and a four-digit station code. As mentioned in section 16.4, they may or may not be preceded by +1, the country code. In practice, however, you have many ways to format a phone number, such as (NNN) NNN-NNNN, NNN-NNN-NNNN, NNN NNN-NNNN, NNN.NNN.NNNN, and NNN NNN NNNN, to name a few. Also, the country code may not be present, may not have a +, and usually (not always) is separated from the number by a space or dash. Whew!

In this lab, your task is to create a phone-number normalizer that takes any of the formats and returns a normalized phone number 1-NNN-NNN-NNNN.

The following are all possible phone numbers:
```
+1 223-456-7890
1-223-456-7890
+1 223 456-7890
(223) 456-7890
1 223 456 7890
223.456.7890
```

**Bonus:** The first digit of the area code and the exchange code can only be 2-9, and the second digit of an area code can't be 9. Use this information to validate the input and return a `ValueError` exception of invalid phone number if the number is invalid.


### Author's solution

In [ ]:
import re

def return_number(match_obj):
    # adds country if needed
    country = match_obj.group("country")
    if not country:
        country = "1"

    # BONUS: raise ValueError if not valid
    if not re.match(r"[2-9][0-8]\d", match_obj.group("area") ):
        raise ValueError("invalid phone number area code {}".format(match_obj.group("area")))
    if not re.match(r"[2-9]\d\d", match_obj.group("exch") ):
        raise ValueError("invalid phone number exchange {}".format(match_obj.group("exch")))

    return(f"{country}-{match_obj.group('area')}-{match_obj.group('exch')}-{match_obj.group('number')}")

# Test
phone_numbers = ["+1 223-456-7890",
                "1-223-456-7890",
                "+1 223 456-7890",
                "(223) 456-7890",
                "1 223 456 7890",
                "999.456.7890",
                "1-989-111-2222",
                "223.456.7890"]

regexp = re.compile(r"\+?(?P<country>\d{1,3})?[- .]?\(?(?P<area>\d{3})\)?[- .]?(?P<exch>(\d{3}))[- .](?P<number>\d{4})")

for number in phone_numbers:
    try:
      normalized_phone_number = regexp.sub(return_number, number)
      print(f'{normalized_phone_number}')
    except ValueError as e:
      print(f'{number} is invalid: {e}')

1-223-456-7890
1-223-456-7890
1-223-456-7890
1-223-456-7890
1-223-456-7890
999.456.7890 is invalid: invalid phone number area code 999
1-989-111-2222 is invalid: invalid phone number exchange 111
1-223-456-7890


### Copilot solution

In [ ]:
# Generated by Copilot
import re

def normalize_phone_number(phone_number):
    phone_number = re.sub(r'\D', '', phone_number)

    if len(phone_number) not in [10, 11]:
        raise ValueError("Invalid phone number length")

    if not re.match(r'^[2-9][0-8][0-9][2-9]', phone_number[-10:]):
        raise ValueError("Invalid area code or exchange code")

    if len(phone_number) == 10:
        phone_number = '1' + phone_number

    phone_number = re.sub(r'(\d{1})(\d{3})(\d{3})(\d{4})', r'\1-\2-\3-\4', phone_number)

    return phone_number

# Test the function.
phone_numbers = ["+1 223-456-7890",
                "1-223-456-7890",
                "+1 223 456-7890",
                "(223) 456-7890",
                "1 223 456 7890",
                "999.456.7890",
                "1-989-111-2222",
                "223.456.7890"]

for number in phone_numbers:
    try:
      normalized_phone_number = normalize_phone_number(number)
      print(f'{normalized_phone_number}')
    except ValueError as e:
      print(f'{number} is invalid: {e}')

1-223-456-7890
1-223-456-7890
1-223-456-7890
1-223-456-7890
1-223-456-7890
999.456.7890 is invalid: Invalid area code or exchange code
1-989-111-2222 is invalid: Invalid area code or exchange code
1-223-456-7890


### Prompt:

```
Using regular expressions, write code to normalize phone numbers in
formats (NNN) NNN-NNNN, NNN-NNN-NNNN, NNN NNN-NNNN, NNN.NNN.NNNN, and NNN NNN
NNNN (with or without a country code and a +) to 1-NNN-NNN-NNNN. Also raise a
ValueError if the first digit of the area code and the exchange code is not
2-9, or the second digit of the area code is 9.
```


### Colaboratory solution

In [ ]:
# prompt: Write the code to normalize phone numbers as specified in the cell above.

import re

def normalize_phone_number(phone_number):
  """
  Normalizes a phone number to the format 1-NNN-NNN-NNNN.

  Args:
    phone_number: The phone number to normalize.

  Returns:
    The normalized phone number.

  Raises:
    ValueError: If the phone number is not in a valid format.
  """

  # Remove all non-digit characters.
  phone_number = re.sub(r'\D', '', phone_number)

  # Check if the phone number is in a valid format.
  if not re.match(r'^(1)?\d{10}$', phone_number):
    raise ValueError('Invalid phone number format.')

  # Check if the first digit of the area code and the exchange code is not 2-9, or the second digit of the area code is 9.
  if not re.match(r'^1?[2-9]\d{2}[2-9]\d{6}$', phone_number):
    raise ValueError('Invalid area code or exchange code.')

  # Normalize the phone number to the format 1-NNN-NNN-NNNN.
  return '-'.join([phone_number[0:1], phone_number[1:4], phone_number[4:7], phone_number[7:]])

# Test the function.
phone_numbers = ["+1 223-456-7890",
                "1-223-456-7890",
                "+1 223 456-7890",
                "(223) 456-7890",
                "1 223 456 7890",
                "999.456.7890",
                "1-989-111-2222",
                "223.456.7890"]

for number in phone_numbers:
    try:
      normalized_phone_number = normalize_phone_number(number)
      print(f'{normalized_phone_number}')
    except ValueError as e:
      print(f'{number} is invalid: {e}')

1-223-456-7890
1-223-456-7890
1-223-456-7890
2-234-567-890
1-223-456-7890
9-994-567-890
1-989-111-2222 is invalid: Invalid area code or exchange code.
2-234-567-890
